In [1]:
from scipy.sparse import csr_matrix
import polars as pl
import implicit

: 

In [55]:
import pandas as pd
df = pd.read_parquet('../data/train_interactions.parquet')
df[:10].to_csv('../data/train_interactions.csv')

In [ ]:
df = pd.read_parquet('../data/items_meta.parquet')
df.to_csv('../data/items_meta.csv')

In [20]:
df = pd.read_parquet('../data/users_meta.parquet')
df.to_csv('../data/users_meta.csv')

In [19]:
train = pl.scan_parquet("../data/train_interactions.parquet")

In [22]:
from pyspark.sql.types import IntegerType

ModuleNotFoundError: No module named 'pyspark'

In [21]:
interaction_matrix = (
    train
    .with_columns(
        (-1).alias("interaction")  # Calculate net interaction score
    )
)

NameError: name 'IntegerType' is not defined

In [16]:
interaction_matrix = interaction_matrix.select("user_id", "item_id", "interaction")

In [17]:
interaction_matrix = interaction_matrix.collect()

In [18]:
interaction_matrix

user_id,item_id,interaction
u32,u32,bool
3810,138979,false
101874,331160,false
150332,73709,false
4982,189745,false
149601,289643,false
…,…,…
10718,93558,false
119164,60206,false
171518,99323,false


In [13]:
interaction_matrix['interaction'].unique()

interaction
u8
0
1
255


In [24]:
train

user_id,item_id,weight
u32,u32,u8
3163,147750,1
113832,306166,1
124909,128209,1
183076,91080,1
156215,289701,1
…,…,…
153247,209536,1
21973,291285,1
129733,183808,1


In [7]:
items_meta = pl.read_parquet("../data/items_meta.parquet")
users_meta = pl.read_parquet("../data/users_meta.parquet")
n_items = items_meta["item_id"].max() + 1
n_users = users_meta["user_id"].max() + 1

In [23]:
n_items

337727

In [36]:
train = csr_matrix((train["weight"],
                    (train["user_id"].to_numpy(),
                     train["item_id"].to_numpy())),
                   shape=(n_users, n_items))

IndexError: Index dimension must be 1 or 2

In [35]:
train.toarray().sum()

np.uint64(513)

In [44]:
model = implicit.als.AlternatingLeastSquares(factors=16,
                                             iterations=10,
                                             regularization=1,
                                             alpha=100,
                                             calculate_training_loss=True)
model.fit(train)

/opt/conda/envs/env/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 10/10 [01:41<00:00, 10.14s/it, loss=0.0062] 


In [45]:
test_pairs = pl.read_csv('../data/test_pairs.csv')
test_pairs

user_id,item_id
i64,i64
1,7363
1,73770
1,75700
1,81204
1,110249
…,…
183403,235097
183403,267411
183403,273646


In [46]:
als_predict = (model.user_factors[test_pairs['user_id']] *
               model.item_factors[test_pairs['item_id']]).sum(axis=1)

In [51]:
min(als_predict)

np.float32(-1.5748558)

In [49]:
test_pairs.with_columns(predict=als_predict).write_csv('sample_submission.csv')